In [ ]:
import json
%

In [ ]:
f = open('data/train-v2.0.json')

In [ ]:

data = json.load(f)

In [ ]:
data.keys()

In [ ]:
data['data'][0].keys()

In [ ]:
len([data['data'][i]['title'] for i in range(len(data['data']))])

In [ ]:
len(data['data'][0]['paragraphs'])

In [ ]:
data['data'][0]['paragraphs'][0].keys()

In [ ]:
data['data'][0]['paragraphs'][0]['context']

In [ ]:
len(data['data'][0]['paragraphs'][0]['qas'])

In [ ]:
[data['data'][0]['paragraphs'][1]['qas'][i]['is_impossible'] for i in range(len(data['data'][0]['paragraphs'][1]['qas']))]

In [ ]:
data['data'][0]['paragraphs'][1]['qas'][0]

In [ ]:
data['data'][0]['paragraphs'][0]['qas']

In [ ]:
data['data'][0]['paragraphs'][0]['context'][269]

In [ ]:
# data['data'][1]

In [ ]:
context_lengths = []

def get_max_context_length():
    for data_point in data['data']:
        for paragraph in data_point['paragraphs']:
            context_lengths.append(len(paragraph['context'].split(' ')))

get_max_context_length()

In [ ]:
import numpy as np

In [ ]:
context_lengths = np.array(context_lengths)

In [ ]:
context_lengths.sort()

In [ ]:
len(context_lengths)

In [ ]:
import csv

In [ ]:
def flatten_squad_data(json_data):
    retval = []
    for data_point in json_data:
        title = data_point['title'].lower()
        for paragraph in data_point['paragraphs']:
            context = paragraph['context'].lower()
            for qas in paragraph['qas']:
                question = qas['question'].lower()
                id_here = qas['id'].lower()
                is_impossible = qas['is_impossible']
                for answer in qas['answers']:
                    text = answer['text'].lower()
                    answer_start = answer['answer_start']
                    start_token = len(context[:answer_start].split())
                    end_token = len(context[:answer_start + len(text)].split())
                    retval.append([title, context, question, id_here, is_impossible, text, start_token, end_token])
    return retval

In [ ]:
f = open('data/dev-v2.0.json')
data = json.load(f)
data = flatten_squad_data(data['data'])

In [ ]:
field_list = ['title', 'context', 'question', 'id_here', 'is_impossible', 'text', 'start_token', 'end_token']
dict(zip(field_list, data[0]))

In [ ]:
len(context[:answer_start].split())

In [ ]:
a = "Hi Hello Hola"
len(a[:9].split())
a.split()[2]

In [ ]:
s = "hello a.,"

In [ ]:
import string
s.strip(string.punctuation)

In [ ]:
a = torch.Tensor([[0,0,0,0,0], [0,0,0,0,0]])
end_tokens = torch.Tensor([1,2]).long()

In [ ]:
import torch.nn as nn

In [ ]:
loss = nn.CrossEntropyLoss()
loss(a,end_tokens)

In [ ]:
a = torch.Tensor([[1,1,1,1], [1,1,1,1]])
b = torch.Tensor([[2,2,2,2], [2,2,2,2]])
torch.cat((a,b), 1)

In [ ]:
indices = [2,3,4,5,5]
token = [1,1,1,10,10]
indices = [indices[index] for index in range(len(indices)) if token[index] < 8]

In [ ]:
indices

In [ ]:
a = [1,2,3]

In [ ]:
def get_next_val(a):
    for val in a:
        yield val

In [ ]:
list(get_next_val(a))

In [ ]:
a = None

In [ ]:
if a is None:
    print(1)

In [ ]:
import torch
a = torch.Tensor([[1,2,3], [4,6,3]])

In [ ]:
max_val = a.max(axis=1).indices

In [ ]:
val = torch.Tensor([]).long()

In [ ]:
torch.cat((val, max_val), 0)

In [ ]:
a = []
b = [1,2,3]

In [ ]:
a += b

In [ ]:
a

In [ ]:
a = [1,2,4]

In [ ]:
b = torch.Tensor([2]).long()

In [ ]:
a[b.item()]

In [ ]:
import json
f = open('data/dev_actual.json')

In [ ]:
data = json.load(f)

In [ ]:
set(list(data.keys()))

In [ ]:
from scripts.utils import *

In [ ]:
data_flattened = flatten_squad_data('data/dev-v2.0.json', train_data = False)

In [ ]:
len(data_flattened)

In [ ]:
data_flattened[0]

In [ ]:
import json
with open('data.txt', 'w') as outfile:
    json.dump(a, outfile)


In [ ]:
import torch
from transformers import *


In [ ]:
!pip install sklearn

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
input_ids = tokenizer.encode(input_text)


In [ ]:
token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]

In [ ]:
start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)  
print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))        

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np

In [ ]:
writer = SummaryWriter()

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

In [ ]:
# From here

In [1]:
from scripts.utils_squad_evaluate import *
from scripts.utils_squad import *
%load_ext autoreload
%autoreload 2

In [2]:
# squad_examples = read_squad_examples('data/dev-v2.0.json', True, True)

In [3]:
# len(squad_examples)

In [4]:
import torch
from transformers import *

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
# features = convert_examples_to_features(squad_examples[:1000], tokenizer=tokenizer, max_seq_length=384, doc_stride=128, max_query_length=64, is_training=False)

In [29]:
dataset, examples, features = load_and_cache_examples('data/dev-v2.0.json', False, tokenizer, evaluate=True)

data/dev-v2.0.json
Loading features from cached file  data/dev


In [15]:
sampler = SequentialSampler(dataset)
train_dataloader = DataLoader(dataset, sampler=sampler, batch_size=3)

In [17]:
batch_here = None
for batch_num, batch in enumerate(train_dataloader):
    batch_here = batch
    break


In [18]:
example_indices = batch_here[3]

In [19]:
example_indices

tensor([0, 1, 2])

In [32]:
int(features[0].unique_id)

1000000000

In [ ]:
import torch
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
all_p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)

In [ ]:
all_start_positions = torch.tensor([f.start_position for f in features], dtype=torch.long)
all_end_positions = torch.tensor([f.end_position for f in features], dtype=torch.long)

In [ ]:
from torch.utils.data import *

In [ ]:
dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                all_start_positions, all_end_positions,
                                all_cls_index, all_p_mask)

In [ ]:
start_bool_tensor = all_start_positions[:5] == 0 
end_bool_tensor = all_end_positions[:5] == 0

In [ ]:
~start_bool_tensor

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

In [ ]:
sampler = RandomSampler(dataset)

In [ ]:
train_dataloader = DataLoader(dataset, sampler=sampler, batch_size=2)

In [ ]:
iterator = tqdm(iter(train_dataloader))
for batch_num, batch in tqdm(enumerate(iterator)):
    print(batch_num)
    

In [ ]:
batch_num = 0
batch_here = None
for batch in train_dataloader:
    batch_num += 1
    batch_here = batch
    break

In [ ]:
batch_here[0].shape

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

In [ ]:
output = model.bert(batch[0], attention_mask=batch[1])

In [ ]:
output[0].shape

In [ ]:
logits = model.qa_outputs(output[0])

In [ ]:
start_logits, end_logits = logits.split(1, dim=-1)

In [ ]:
start_logits = start_logits.squeeze(-1)

In [ ]:
start_logits.shape

In [ ]:
import torch 
from torch.utils.data import *

In [ ]:
dataset = TensorDataset(torch.Tensor([1,2,3]), torch.Tensor([4,5,6]))

In [ ]:
sampler = SequentialSampler(dataset)

In [ ]:
train_dataloader = DataLoader(dataset, sampler=sampler, batch_size=1)

In [ ]:
for batch in train_dataloader:
    print(batch)

In [ ]:
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')